In [1]:
import torch
import torchvision
import torchvision.transforms as transforms



In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assume that we are on a CUDA machine, then this should print a CUDA device:

print(device)

cuda:0


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,16,5)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(5*5*16,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x= x.view(-1,5*5*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001,momentum = 0.9)


for epoch in range(2):
    for i,data in enumerate(trainloader,0):
        inputs,labels = data
       
        optimizer.zero_grad()
        outputs = net(inputs)
        print(labels)
        loss = criterion(outputs,labels)
        
        loss.backward()
        optimizer.step()
        print('epoch : %d , batch: %5d , loss: %.3f'% (epoch+1 , i+1 , loss.item()))

print('training done')




G:\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[ 0.1040,  0.1072,  0.1053,  0.0952,  0.0978,  0.1066,  0.1049,
          0.0907,  0.0995,  0.0887],
        [ 0.1037,  0.1080,  0.1047,  0.0947,  0.0974,  0.1052,  0.1063,
          0.0915,  0.1001,  0.0885],
        [ 0.1044,  0.1074,  0.1040,  0.0952,  0.0985,  0.1058,  0.1051,
          0.0908,  0.1000,  0.0888],
        [ 0.1041,  0.1076,  0.1044,  0.0960,  0.0972,  0.1057,  0.1048,
          0.0906,  0.1004,  0.0892]])
epoch : 1 , batch:     1 , loss: 2.303
tensor([[ 0.1044,  0.1083,  0.1052,  0.0949,  0.0978,  0.1058,  0.1052,
          0.0905,  0.0989,  0.0890],
        [ 0.1045,  0.1080,  0.1046,  0.0949,  0.0976,  0.1062,  0.1046,
          0.0905,  0.1000,  0.0892],
        [ 0.1038,  0.1078,  0.1048,  0.0956,  0.0978,  0.1061,  0.1047,
          0.0909,  0.1000,  0.0885],
        [ 0.1034,  0.1085,  0.1057,  0.0950,  0.0970,  0.1058,  0.1053,
          0.0911,  0.0998,  0.0883]])
epoch : 1 , batch:     2 , loss: 2.302
tensor([[ 0.1042,  0.1084,  0.1053,  0.0946,  0.

In [6]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


G:\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 40 %


In [ ]:
##Name to Language predictor :


In [3]:
#data preprocessing
import os

c =0 
categories = []
names =[]
dict = {}
directory = os.fsencode("./data/names/")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    f_o = open("./data/names/{}".format(filename),"r",encoding = 'utf-8')
    x = f_o.read().strip().split('\n')
    names.append(x[:])
    categories.append( [c]*len(x) )
    dict[c] = filename
    c+=1

names = [j for i in names for j in i]
categories = [j for i in categories for j in i]

import unicodedata
for i in range(20074):
    names[i] = unicodedata.normalize('NFD', names[i]).encode('ascii','ignore')
    names[i] = names[i].decode('utf-8')


G:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead


In [2]:
import random
names_ascii = []

for w in names:
    arr = [ord(c) for c in w]
    names_ascii.append(arr)
len(names_ascii[1])

combined = list(zip(names_ascii, categories))
random.shuffle(combined)

names_ascii[:], categories[:] = zip(*combined)


import itertools
import numpy as np
names_ascin = np.array(list(itertools.zip_longest(*names_ascii, fillvalue=0)) ,dtype =np.int64).T
categories_n = np.array(categories,dtype = np.int64)


In [4]:
import torch
from torch.autograd import Variable
x_train = torch.LongTensor(torch.from_numpy(names_ascin[:19000])) 
x_test =  torch.LongTensor(torch.from_numpy(names_ascin[19000:])) 
y_train = Variable(torch.LongTensor(torch.from_numpy(categories_n[:19000])) )
y_test =  Variable(torch.LongTensor(torch.from_numpy(categories_n[19000:])) )



In [5]:

import torch.nn as nn
import torch.optim as optim


class RNNClassifier(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,num_layers):
        super(RNNClassifier,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size,num_layers)
        self.linear = nn.Linear(hidden_size ,output_size)
        
    def forward(self,inp):
        
        batch_size = inp.size(0)
        inp  = inp.t()
        embedded = self.embedding(inp)
        hidden = Variable(torch.zeros(self.num_layers,batch_size,self.hidden_size))
        output,hidden = self.gru(embedded,hidden)
        fc_output = self.linear(hidden)
        
        return fc_output
    
classifier = RNNClassifier(128,100,18,1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(),lr = 0.001)

for epoch in range(300):
    outt = classifier(x_train)
   
    loss = criterion(outt[0],y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("epoch : %d , loss : %.3f"% (epoch+1, loss.item()))
        
print("Done")     
      
  

epoch : 1 , loss : 2.852
epoch : 2 , loss : 2.645
epoch : 3 , loss : 2.460
epoch : 4 , loss : 2.300
epoch : 5 , loss : 2.171
epoch : 6 , loss : 2.074
epoch : 7 , loss : 2.005
epoch : 8 , loss : 1.957
epoch : 9 , loss : 1.922
epoch : 10 , loss : 1.898
epoch : 11 , loss : 1.881
epoch : 12 , loss : 1.872
epoch : 13 , loss : 1.867
epoch : 14 , loss : 1.865
epoch : 15 , loss : 1.863
epoch : 16 , loss : 1.860
epoch : 17 , loss : 1.856
epoch : 18 , loss : 1.851
epoch : 19 , loss : 1.848
epoch : 20 , loss : 1.844
epoch : 21 , loss : 1.841
epoch : 22 , loss : 1.837
epoch : 23 , loss : 1.833
epoch : 24 , loss : 1.828
epoch : 25 , loss : 1.823
epoch : 26 , loss : 1.818
epoch : 27 , loss : 1.813
epoch : 28 , loss : 1.808
epoch : 29 , loss : 1.802
epoch : 30 , loss : 1.797
epoch : 31 , loss : 1.790
epoch : 32 , loss : 1.783
epoch : 33 , loss : 1.776
epoch : 34 , loss : 1.768
epoch : 35 , loss : 1.759
epoch : 36 , loss : 1.750
epoch : 37 , loss : 1.741
epoch : 38 , loss : 1.731
epoch : 39 , loss : 1

In [72]:
with torch.no_grad():
    
    tout = classifier(x_test)
    
    label_p =torch.argmax(nn.functional.softmax(tout[0],dim = 1) ,dim = 1)
    print(y_test.size(0))
    print((y_test == label_p).sum().item())
    print((y_test == label_p).sum().item()/y_test.size(0))

1074
870
0.8100558659217877


In [77]:
def pred(name):
    classifier = RNNClassifier(128,100,18,1)
    classifier.load_state_dict(torch.load('./data/lang_name'))
    name = unicodedata.normalize('NFD', name).encode('ascii','ignore')
    name = name.decode('utf-8')
    arr = [ord(c) for c in name]
    if len(arr) >= 20 :
        arr = arr[:20]
    else:
        for i in range(len(arr),20):
            arr.append(0)
        
    
    arr1 = []
    arr1.append(arr)
    arr1 = np.array(arr1,dtype = np.int64)
    nam_t = torch.LongTensor(torch.from_numpy(arr1))
    print(nn.functional.softmax(classifier(nam_t)[0]))
    label_p = torch.argmax(nn.functional.softmax(classifier(nam_t)[0],dim = 1) ,dim = 1)
    print(dict[label_p.item()])
    
pred('Ayub')

tensor([[ 0.9811,  0.0036,  0.0007,  0.0001,  0.0047,  0.0002,  0.0011,
          0.0006,  0.0006,  0.0004,  0.0020,  0.0017,  0.0002,  0.0005,
          0.0010,  0.0005,  0.0003,  0.0010]])
Arabic.txt


G:\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [30]:
torch.save(classifier.state_dict(),'./data/lang_name')